<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_5_10AM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml
!pip install parse

    100% |████████████████████████████████| 5.8MB 6.6MB/s 
  Running setup.py bdist_wheel for parse ... - done
  Stored in directory: /root/.cache/pip/wheels/ef/db/c6/18568a2cc574848f3996ac4552241fbec046b7be29feb2077d
Successfully built parse


In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 16 #limit to check whether they are relative or not
NOISE = 9

In [3]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 백석역
2 -> 진학사
3 -> 수능 등급컷
4 -> 양수경 남편
5 -> 홍현희
6 -> 한국교육과정평가원
7 -> 2019 수능 등급컷
8 -> 수능
9 -> 수능 성적 발표
10 -> 황교익
11 -> 처음학교로
12 -> 이설
13 -> 블레이드앤소울 레볼루션
14 -> 한정수
15 -> 하춘화나이
16 -> 블랭크코퍼레이션
17 -> 메가스터디
18 -> 이일재
19 -> 이선정
20 -> 로맨스는 별책부록


In [4]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
 
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

백석역  and  수능
진학사  and  수능 등급컷
진학사  and  수능
수능 등급컷  and  2019 수능 등급컷
수능 등급컷  and  수능
양수경 남편  and  수능
양수경 남편  and  한정수
홍현희  and  수능
한국교육과정평가원  and  수능 등급컷
한국교육과정평가원  and  수능
2019 수능 등급컷  and  수능 등급컷
2019 수능 등급컷  and  수능
수능 성적 발표  and  수능 등급컷
수능 성적 발표  and  수능
황교익  and  수능
처음학교로  and  수능
이설  and  수능
블레이드앤소울 레볼루션  and  수능
한정수  and  수능
하춘화나이  and  수능
블랭크코퍼레이션  and  수능
메가스터디  and  수능
이일재  and  수능
이선정  and  수능
로맨스는 별책부록  and  수능


In [5]:
print('\n',label,'\n')

print(matrix)

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])

print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

[[0, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 0, 28, 0, 2, 6, 15, 101, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 9, 0, 0, 2, 8, 18, 182, 7, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0, 33, 0, 0, 0, 2, 0, 0], [4, 0, 9, 0, 0, 0, 0, 27, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 3, 28, 0, 2, 0, 9, 96, 3, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 10, 92, 0, 2, 8, 0, 219, 7, 0, 0, 2, 0, 2, 0, 0, 3, 2, 0, 0], [4, 2, 13, 0, 2, 9, 3, 0, 5, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0], [4, 2, 22, 0, 2, 5, 8, 200, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0], [4, 0, 9, 2, 2, 0, 0, 27, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0], [4, 0, 9, 0, 2, 0, 0, 27, 0, 0, 0, 2, 0,

In [6]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  백석역 | 진학사 | 수능 등급컷 | 양수경 남편 | 홍현희 | 한국교육과정평가원 | 2019 수능 등급컷 | 수능 | 수능 성적 발표 | 황교익 | 처음학교로 | 이설 | 블레이드앤소울 레볼루션 | 한정수 | 하춘화나이 | 블랭크코퍼레이션 | 메가스터디 | 이일재 | 이선정 | 로맨스는 별책부록
